## S.6 RDD API
### 2. RDD 사용하기

In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
nRdd = spark.sparkContext.parallelize([1, 2, 3, 4])
squared = nRdd.map(lambda x: x * x)
print (squared)

PythonRDD[1] at RDD at PythonRDD.scala:53


In [3]:
#변환의 실제 결과를 보려면 collect()를 사용해서 출력해야 한다.
print (squared.collect())

[1, 4, 9, 16]


In [4]:
%%writefile ./data/ds_spark_2cols.csv
35, 2
40, 27
12, 38
15, 31
21, 1
14, 19
46, 1
10, 34
28, 3
48, 1
16, 2
30, 3
32, 2
48, 1
31, 2
22, 1
12, 3
39, 29
19, 37
25, 2

Overwriting ./data/ds_spark_2cols.csv


In [5]:
import os
myRdd4 = spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_2cols.csv"))

myRdd4.take(5)

#여기서는 ''"" 딱히 구별 안함, 문자로 읽어왔다는 이야기

['35, 2', '40, 27', '12, 38', '15, 31', '21, 1']

In [6]:
#2차원 list로
myRdd5 = myRdd4.map(lambda line: line.split(','))
myRdd5.take(5)

[['35', ' 2'], ['40', ' 27'], ['12', ' 38'], ['15', ' 31'], ['21', ' 1']]

In [7]:
'35'+'2' #아직은 문자다 '

'352'

#### 문자열을 정수로 변환

In [8]:
#for 문 이용시
x=['35', '2']
for i in x:
    print(i)

35
2


In [9]:
#for 문 이용시, list로
x=['35', '2']
y=list()
for i in x:
    y.append(int(i))
print(y)

[35, 2]


In [10]:
# 줄여서 처리
x=['35', '2']
[int(i) for i in x] #pythonic

[35, 2]

In [11]:
#map 함수에 넣어서 정수로 만들기
myRdd6 = myRdd5.map(lambda x: [int(i) for i in x])
myRdd6.take(5)

[[35, 2], [40, 27], [12, 38], [15, 31], [21, 1]]

#### txt파일 생성

In [12]:
%%writefile data/ds_spark_wiki.txt
Wikipedia
Apache Spark is an open source cluster computing framework.
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
Apache Spark Apache Spark Apache Spark Apache Spark
아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
Originally developed at the University of California, Berkeley's AMPLab,
the Spark codebase was later donated to the Apache Software Foundation,
which has maintained it since.
Spark provides an interface for programming entire clusters with
implicit data parallelism and fault-tolerance.

Overwriting data/ds_spark_wiki.txt


#### 단어로 분리하기

In [13]:
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))

In [14]:
# 문서를 문장으로 쪼개기, 사용자함수 사용 x

sentences=myRdd2.map(lambda x:x.split()) #split()안에 인자가 없으면 whitespace로 분리

In [15]:
# 문장 count
sentences.count()

10

#### 사용자 함수를 이용한 단어 분리

In [16]:
#사용자 함수 이용

def mySplit(x):
    return x.split()

sentences2=myRdd2.map(mySplit)
sentences2.count()

10

In [17]:
sentences.take(3)  # 3: 개수, 2차원 배열로 되어있는 것 확인 가능

[['Wikipedia'],
 ['Apache',
  'Spark',
  'is',
  'an',
  'open',
  'source',
  'cluster',
  'computing',
  'framework.'],
 ['아파치', '스파크는', '오픈', '소스', '클러스터', '컴퓨팅', '프레임워크이다.']]

In [18]:
#document를 sentence로 쪼갬
for line in sentences.collect():  #sentences.collect() : 전체 data
    
    # sentence를 word로 쪼갬
    for word in line:
        print (word, end=" ")
    print ("\n-----")

Wikipedia 
-----
Apache Spark is an open source cluster computing framework. 
-----
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. 
-----
Apache Spark Apache Spark Apache Spark Apache Spark 
-----
아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 
-----
Originally developed at the University of California, Berkeley's AMPLab, 
-----
the Spark codebase was later donated to the Apache Software Foundation, 
-----
which has maintained it since. 
-----
Spark provides an interface for programming entire clusters with 
-----
implicit data parallelism and fault-tolerance. 
-----


#### 문자 개수

In [19]:
#len() : 문자 개수를 센다
len("Apache Spark is an open source cluster computing framework.")

59

In [20]:
#문자 개수를 센 후 배열로 만들어보기
#문장별로 센다
myRdd2.map(lambda s:len(s)).collect()

[9, 59, 32, 51, 31, 72, 71, 30, 64, 46]

#### 교체

In [21]:
myList=["this is","a line"]
_rdd=spark.sparkContext.parallelize(myList) #list에서 rdd 만들기

In [22]:
wordsRdd=_rdd.map(lambda x:x.split())
print (wordsRdd.collect())

[['this', 'is'], ['a', 'line']]


In [23]:
repRdd=_rdd.map(lambda x:x.replace("a","AA"))
repRdd.take(10)

['this is', 'AA line']

#### 대소문자 변환

In [24]:
's'.upper()

'S'

In [25]:
upperRDD =wordsRdd.map(lambda x: x[0].upper())
print (upperRDD.collect())

['THIS', 'A']


In [26]:
upper2RDD =wordsRdd.map(lambda x: [i.upper() for i in x])
print (upper2RDD.collect())

[['THIS', 'IS'], ['A', 'LINE']]


#### reduce

In [27]:
myRdd100 = spark.sparkContext.parallelize(range(1,101)) #1~100숫자 생성
myRdd100.reduce(lambda subtotal, x: subtotal + x)

5050

#### 단순 통계 기능

In [28]:
print ("sum: ", myRdd100.sum())
print ("min: ", myRdd100.min())
print ("max: ", myRdd100.max())
print ("standard deviation:", myRdd100.stdev()) ##표준편차
print ("variance: ", myRdd100.variance())       ##분산

sum:  5050
min:  1
max:  100
standard deviation: 28.86607004772212
variance:  833.25


#### filter()

In [29]:
#line을 받아서 line안에 spark가 있는지 없는지 보기
myRdd_spark=myRdd2.filter(lambda line: "Spark" in line) 
print ("How many lines having 'Spark': ",myRdd_spark.count())

How many lines having 'Spark':  4


In [30]:
myRdd_unicode = myRdd2.filter(lambda line: u"스파크" in line)
print (myRdd_unicode.first())

아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.


#### filter()를 이용하여 stopwords 제거하기
- stopwords: 불용어, '이', '그', '저' 등

In [31]:
# 단어 기준, 단어가 stopwords에 있지 않으면 true가 되어서 아래서 출력
stopwords = ['is','am','are','the','for','a', 'an', 'at']

myRdd_stop = myRdd2.flatMap(lambda x:x.split())\
                    .filter(lambda x: x not in stopwords)

In [32]:
for words in myRdd_stop.collect(): # collect(): 전체 단어
    print (words, end=' ')

Wikipedia Apache Spark open source cluster computing framework. 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. Apache Spark Apache Spark Apache Spark Apache Spark 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 Originally developed University of California, Berkeley's AMPLab, Spark codebase was later donated to Apache Software Foundation, which has maintained it since. Spark provides interface programming entire clusters with implicit data parallelism and fault-tolerance. 

#### foreach()
- 반환 값이 없기 때문에 아무것도 안나온다

In [33]:
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(lambda x: x + 1)

In [34]:
#map은 있다 (foreach()와 비교용)
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).map(lambda x: x + 1).collect()

[2, 3, 4, 5, 6]

In [35]:
def f(x): print(x)
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(f) #foreach: 각각의 요소에 대해서

#### pipeline
- 파이프라인은 transformation(예: map()), action(예: collect()) 함수를 연이어 적용하는 방식

In [36]:
upper2list=wordsRdd.map(lambda x: [i.upper() for i in x]).collect()
print (type(upper2list))
print(upper2list)

<class 'list'>
[['THIS', 'IS'], ['A', 'LINE']]


In [37]:
wordsLength = wordsRdd\
    .map(len)\
    .collect()
print (wordsLength)

[2, 2]


#### 파일에 쓰기

In [39]:
#폴더 생성
spark.sparkContext.parallelize(upper2list).saveAsTextFile("data/ds_spark_wiki_out")

In [41]:
!dir data\ds_spark_wiki_out

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: C826-817A

 C:\Users\user\Bigdata\data\ds_spark_wiki_out 디렉터리

2020-09-19  오후 06:41    <DIR>          .
2020-09-19  오후 06:41    <DIR>          ..
2020-09-19  오후 06:41                12 .part-00000.crc
2020-09-19  오후 06:41                 8 ._SUCCESS.crc
2020-09-19  오후 06:41                29 part-00000
2020-09-19  오후 06:41                 0 _SUCCESS
               4개 파일                  49 바이트
               2개 디렉터리  373,071,044,608 바이트 남음


In [42]:
_rdd=spark.sparkContext.textFile("data/ds_spark_wiki_out")
_rdd.collect()

["['THIS', 'IS']", "['A', 'LINE']"]

In [43]:
# 파일로 쓰기 : coalesce()함수
_rdd.map(lambda x: "".join(x)).coalesce(1).saveAsTextFile("data/ds_spark_wiki_txt")

In [46]:
!type data\ds_spark_wiki_txt\part-00000

['THIS', 'IS']
['A', 'LINE']


#### groupby
- 문자 데이터 처리 가능

In [49]:
myRdd2.take(10)

['Wikipedia',
 'Apache Spark is an open source cluster computing framework.',
 '아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.',
 'Apache Spark Apache Spark Apache Spark Apache Spark',
 '아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크',
 "Originally developed at the University of California, Berkeley's AMPLab,",
 'the Spark codebase was later donated to the Apache Software Foundation,',
 'which has maintained it since.',
 'Spark provides an interface for programming entire clusters with',
 'implicit data parallelism and fault-tolerance.']

In [51]:
#myRdd_group=myRdd2.flatMap(lambda x:x.split()).groupBy(lambda x:w[0:2])
myRdd_group=myRdd2.groupBy(lambda x:x[0:2])  # 앞글자 2개만 떼어서 groupby
# Wikipidia에서 Wi만 뜯어지게 된다

#각 문장의 맨 앞 2글자만 떼어서
for (k,v) in myRdd_group.collect(): # k,v: key, value // key, value 쌍으로 되어있다.
    print ("{}: {}".format(k, v))
    #ResultIterable object: value가 여러개, 반복문이 하나 더 필요하다!

Wi: <pyspark.resultiterable.ResultIterable object at 0x0000027D30AEB088>
Ap: <pyspark.resultiterable.ResultIterable object at 0x0000027D30AEB3C8>
아파: <pyspark.resultiterable.ResultIterable object at 0x0000027D30AEB848>
Or: <pyspark.resultiterable.ResultIterable object at 0x0000027D30AEB608>
th: <pyspark.resultiterable.ResultIterable object at 0x0000027D30AEBB88>
wh: <pyspark.resultiterable.ResultIterable object at 0x0000027D30AEBF48>
Sp: <pyspark.resultiterable.ResultIterable object at 0x0000027D30AEBBC8>
im: <pyspark.resultiterable.ResultIterable object at 0x0000027D30AEB408>


In [52]:
#myRdd_group=myRdd2.flatMap(lambda x:x.split()).groupBy(lambda x:w[0:2])
myRdd_group=myRdd2.groupBy(lambda x:x[0:2])

for (k,v) in myRdd_group.collect():
    for eachValue in v:
        print ("{}: {}".format(k, eachValue))
    print ("-----")

Wi: Wikipedia
-----
Ap: Apache Spark is an open source cluster computing framework.
Ap: Apache Spark Apache Spark Apache Spark Apache Spark
-----
아파: 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
아파: 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
-----
Or: Originally developed at the University of California, Berkeley's AMPLab,
-----
th: the Spark codebase was later donated to the Apache Software Foundation,
-----
wh: which has maintained it since.
-----
Sp: Spark provides an interface for programming entire clusters with
-----
im: implicit data parallelism and fault-tolerance.
-----


In [53]:
_testList=[("Seoul",1),("Seoul",1),("Seoul",1),("Busan",1),("Busan",1),
           ("Seoul",1),("Busan",1),
           ("Seoul",1),("Seoul",1),("Busan",1),("Busan",1)]

In [54]:
#Rdd 생성
_testRdd=spark.sparkContext.parallelize(_testList)

In [55]:
_testRdd.groupBy(lambda x:x[0]).collect()

[('Seoul', <pyspark.resultiterable.ResultIterable at 0x27d30b05508>),
 ('Busan', <pyspark.resultiterable.ResultIterable at 0x27d30b054c8>)]

In [56]:
#mapValues 이용해서 출력가능!
_testRdd.groupBy(lambda x:x[0]).mapValues(lambda x: list(x)).collect()

[('Seoul',
  [('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1)]),
 ('Busan',
  [('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1)])]

In [57]:
#lambda말고 list로도가능!
_testRdd.groupBy(lambda x:x[0]).mapValues(list).collect()

[('Seoul',
  [('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1)]),
 ('Busan',
  [('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1)])]